In [1]:
data_path = "./Data/starting_ki/train_all_tasks.csv"

In [2]:
import pandas as pd
data = pd.read_csv(data_path)
data = data[data["label_category"]!="none"]

In [4]:
label_category_map = {
    "1. threats, plans to harm and incitement":"Threat",
    "2. derogation":"Derogation",
    "3. animosity":"Animosity",
    "4. prejudiced discussions":"Prejudiced discussion",
    "Threat":"1. threats, plans to harm and incitement",
    "Derogation":"2. derogation",
    "Animosity":"3. animosity",
    "Prejudiced discussion":"4. prejudiced discussions"
}
print(label_category_map["4. prejudiced discussions"])

Prejudiced discussion


In [26]:
import random
# function returns two random text samples to use them as GPT-J prompt
def get_three_random_samples(data_frame):
    # get two random samples from `data file`
    s1, s2, s3 = random.sample(range(0, len(data)), 3)
    print(s1,s2,s3)
    # get text labels
    label1 = label_category_map[data_frame['label_category'].iloc[s1]]
    label2 = label_category_map[data_frame['label_category'].iloc[s2]]
    label3 = label_category_map[data_frame['label_category'].iloc[s3]]
#     print(label_category_map[label1])
    
    # return text1, label1, text2, label2
    return data_frame['text'].iloc[s1], label1, data_frame['text'].iloc[s2], label2, data_frame['text'].iloc[s3], label3


In [41]:
def get_prompt(text1, label1, text2, label2, text3, label3):
    # define a function that takes as input three samples and generates the prompt
    # that we should pass to the GPT-3 language model for completion.
    description = "Each item in the following list contains a sexist tweet and the respective category. Category is one of 'Threat', 'Derogation', 'Animosity', or 'Prejudiced discussion'"
    prompt = (f"{description}\n"
            f"Tweet: {text1} (Sentiment: {label1})\n"
            f"Tweet: {text2} (Sentiment: {label2})\n"
            f"Tweet: {text3} (Sentiment: {label3})\n"
            f"Tweet:")
    return prompt


In [42]:
a,b,c,d,e,f=get_three_random_samples(data)
get_prompt(a,b,c,d,e,f)

1695 2109 2126


"Each item in the following list contains a sexist tweet and the respective category. Category is one of 'Threat', 'Derogation', 'Animosity', or 'Prejudiced discussion'\nTweet: It was rape because she regretted it afterwards, duh. There's no way she could have sex with a nerd. Sheesh. (Sentiment: Prejudiced discussion)\nTweet: Lol guys, I seen the original, it's posted by a mum xD even the mum is a whore and won't tell her daughter off. (Sentiment: Derogation)\nTweet: HAHAHA! Why would an alpha get married to an old hag when he has literally hundreds of hot 18-19 year olds (and older bitches too but eeewww amirite??) begging him to fuck them? Feeeemale logic. (Sentiment: Derogation)\nTweet:"

In [29]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large")
model = GPT2LMHeadModel.from_pretrained("gpt2-large", pad_token_id=tokenizer.eos_token_id)


In [36]:
prompt_list=[]
res_list=[]

In [ ]:
a,b,c,d,e,f=get_three_random_samples(data)
sentence = get_prompt(a,b,c,d,e,f)
prompt_list.append(sentence)
print(sentence)
print("===================================")
input_ids = tokenizer.encode(sentence, return_tensors='pt')
# generate text until the output length (which includes the context length) reaches 50
output = model.generate(input_ids, max_length=250, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
ans=tokenizer.decode(output[0], skip_special_tokens=True)
print(ans)
print("===================================")
                                                            
res_list.append(ans)

2338 2013 2852
Each item in the following list contains a sexist tweet and the respective category. Category is one of 'Threat', 'Derogation', 'Animosity', or 'Prejudiced discussion'
Tweet: if women want men to be responsible for babies they make they shouldnt make them unless married (Sentiment: Animosity)
Tweet: To think all women are trying to repress this feeling... that is female over-eagerness. (Sentiment: Derogation)
Tweet: [URL] This cunt is Antifa. (Sentiment: Animosity)
Tweet:
